In [1]:
import pandas as pd

Model Training

In [8]:
df = pd.read_csv('gemstone.csv')

In [9]:
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,1,0.30,Ideal,E,SI1,62.1,58.0,4.27,4.29,2.66,499
1,2,0.33,Premium,G,IF,60.8,58.0,4.42,4.46,2.70,984
2,3,0.90,Very Good,E,VVS2,62.2,60.0,6.04,6.12,3.78,6289
3,4,0.42,Ideal,F,VS1,61.6,56.0,4.82,4.80,2.96,1082
4,5,0.31,Ideal,F,VVS1,60.4,59.0,4.35,4.43,2.65,779


In [10]:
df = df.drop(labels=['id'],axis=1)

In [11]:
df

,carat,cut,color,clarity,depth,table,x,y,z,price
0,0.30,Ideal,E,SI1,62.1,58.0,4.27,4.29,2.66,499
1,0.33,Premium,G,IF,60.8,58.0,4.42,4.46,2.70,984
2,0.90,Very Good,E,VVS2,62.2,60.0,6.04,6.12,3.78,6289
3,0.42,Ideal,F,VS1,61.6,56.0,4.82,4.80,2.96,1082
4,0.31,Ideal,F,VVS1,60.4,59.0,4.35,4.43,2.65,779
...,...,...,...,...,...,...,...,...,...,...
26962,1.11,Premium,G,SI1,62.3,58.0,6.61,6.52,4.09,5408
26963,0.33,Ideal,H,IF,61.9,55.0,4.44,4.42,2.74,1114
26964,0.51,Premium,E,VS2,61.7,58.0,5.12,5.15,3.17,1656
26965,0.27,Very Good,F,VVS2,61.8,56.0,4.19,4.20,2.60,682


In [18]:
## Independent and dependent features
X = df.drop(labels=['price'],axis=1)
Y = df[['price']]
print(X)
print(Y)

       carat        cut color clarity  depth  table     x     y     z
0       0.30      Ideal     E     SI1   62.1   58.0  4.27  4.29  2.66
1       0.33    Premium     G      IF   60.8   58.0  4.42  4.46  2.70
2       0.90  Very Good     E    VVS2   62.2   60.0  6.04  6.12  3.78
3       0.42      Ideal     F     VS1   61.6   56.0  4.82  4.80  2.96
4       0.31      Ideal     F    VVS1   60.4   59.0  4.35  4.43  2.65
...      ...        ...   ...     ...    ...    ...   ...   ...   ...
26962   1.11    Premium     G     SI1   62.3   58.0  6.61  6.52  4.09
26963   0.33      Ideal     H      IF   61.9   55.0  4.44  4.42  2.74
26964   0.51    Premium     E     VS2   61.7   58.0  5.12  5.15  3.17
26965   0.27  Very Good     F    VVS2   61.8   56.0  4.19  4.20  2.60
26966   1.25    Premium     J     SI1   62.0   58.0  6.90  6.88  4.27

[26967 rows x 9 columns]
       price
0        499
1        984
2       6289
3       1082
4        779
...      ...
26962   5408
26963   1114
26964   1656
2696

In [19]:
# Define which columns should be ordinal-encoded and which should be scaled
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [20]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [37]:
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [38]:
## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())

    ]

)

# Categorigal Pipeline
cat_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
    ('scaler',StandardScaler())
    ]

)

preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_cols),
('cat_pipeline',cat_pipeline,categorical_cols)
])

In [39]:
## Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=30)

In [40]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [41]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,0.561674,0.318936,-0.651005,0.704838,0.692859,0.723432,0.983583,0.226283,0.576148
1,-0.604087,-0.397691,1.589878,-0.533052,-0.486169,-0.533329,-0.812066,-0.945547,-0.639724
2,-0.520818,-0.254365,-1.099182,-0.409263,-0.376688,-0.410385,0.983583,0.812198,-1.247659
3,-0.999613,0.247274,-1.099182,-1.178523,-1.134635,-1.107067,0.983583,0.812198,0.576148
4,0.228599,0.963901,-0.651005,0.413049,0.330729,0.477544,-0.812066,0.812198,-0.031788


In [42]:
## Model Training

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [43]:
regression=LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [ ]:
LinearRegression()
In a Jupyter environment, please rerun this cell to show the HTML representation or trust the notebook.
On GitHub, the HTML representation is unable to render, please try loading this page with nbviewer.org.

LinearRegression()

In [45]:
regression.coef_

array([[4989.90118006, -117.54738915,  -71.5693016 , -811.65093156,
          23.86496768,  -22.98019608,  128.63802707, -551.41625636,
         835.50098553]])

In [46]:
regression.intercept_

array([3944.254662])

In [47]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [48]:
## Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE: 1179.6445914801739
MAE: 813.8092266859256
R2 score 91.44173860508148


Lasso
Model Training Performance
RMSE: 1180.8351588815315
MAE: 815.4198090362224
R2 score 91.42445487594959


Ridge
Model Training Performance
RMSE: 1179.823203867093
MAE: 814.0750740594759
R2 score 91.43914676134862


Elasticnet
Model Training Performance
RMSE: 1629.2689629476954
MAE: 1079.23871584633
R2 score 83.6744149366052




In [49]:
model_list

['LinearRegression', 'Lasso', 'Ridge', 'Elasticnet']